In [3]:
from collections import Counter
import configparser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI

def get_twitter(config_file):
    """ Construct an instance of TwitterAPI using the tokens you entered above.
    Returns:
      An instance of TwitterAPI.
    """
    
    config = configparser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                       config.get('twitter', 'consumer_key'),
                       config.get('twitter', 'consumer_secret'),
                       config.get('twitter', 'access_token'),
                       config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [99]:
type(twitter)

TwitterAPI.TwitterAPI.TwitterAPI

In [300]:
request = twitter.request('users/lookup', {'screen_name': 'poojitha_vd'}) 
request.headers

{'content-length': '1305', 'x-twitter-response-tags': 'BouncerCompliant', 'content-disposition': 'attachment; filename=json.json', 'pragma': 'no-cache', 'x-xss-protection': '1; mode=block', 'expires': 'Tue, 31 Mar 1981 05:00:00 GMT', 'x-rate-limit-reset': '1473282697', 'server': 'tsa_b', 'x-connection-hash': '1da194f72e9acb29f328d9bc3a37730e', 'x-rate-limit-limit': '180', 'status': '200 OK', 'set-cookie': 'lang=en; Path=/, guest_id=v1%3A147328179729894416; Domain=.twitter.com; Path=/; Expires=Fri, 07-Sep-2018 20:56:37 UTC', 'date': 'Wed, 07 Sep 2016 20:56:37 GMT', 'x-access-level': 'read-write-directmessages', 'cache-control': 'no-cache, no-store, must-revalidate, pre-check=0, post-check=0', 'last-modified': 'Wed, 07 Sep 2016 20:56:37 GMT', 'x-content-type-options': 'nosniff', 'strict-transport-security': 'max-age=631138519', 'x-response-time': '32', 'x-rate-limit-remaining': '179', 'content-type': 'application/json;charset=utf-8', 'x-tsa-request-body-time': '0', 'x-frame-options': 'SA

In [304]:
for r in request:
    print(r)

{'profile_background_image_url': None, 'url': None, 'has_extended_profile': True, 'profile_link_color': '2B7BB9', 'profile_sidebar_border_color': 'C0DEED', 'favourites_count': 0, 'time_zone': None, 'verified': False, 'default_profile_image': False, 'default_profile': True, 'profile_background_color': 'F5F8FA', 'id': 770765409721184256, 'is_translation_enabled': False, 'friends_count': 63, 'protected': False, 'entities': {'description': {'urls': []}}, 'created_at': 'Tue Aug 30 23:29:28 +0000 2016', 'id_str': '770765409721184256', 'profile_text_color': '333333', 'following': True, 'profile_background_tile': False, 'screen_name': 'Poojitha_VD', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/770765409721184256/1472600441', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/770767925464690688/62UrUaMg_normal.jpg', 'profile_image_url': 'http://pbs.twimg.com/profile_images/770767925464690688/62UrUaMg_normal.jpg', 'lang': 'en'

In [101]:
request.status_code

200

In [102]:
tweets = [r for r in request]
print('found %d tweets' % len(tweets))

found 15 tweets


In [103]:
a_tweet = tweets[0]
type(a_tweet)

dict

In [164]:
user = a_tweet['user']
type(user)

dict

In [198]:
screen_name = user['screen_name']
request = twitter.request('followers/list', {'screen_name': 'poojitha_vd', 'count':200})
followers = [r for r in request]
print(len(followers))

2


In [51]:
def read_screen_names(filename):
    with open(filename) as f:
        lines = [line.rstrip('\n') for line in open(filename)]
        return list(lines)
    
read_screen_names('candidates.txt')

['DrJillStein', 'GovGaryJohnson', 'HillaryClinton', 'realDonaldTrump']

In [138]:
request = twitter.request('users/show', {'screen_name':'realDonaldTrump'}) 
print(type(request))
request.status_code

<class 'TwitterAPI.TwitterAPI.TwitterResponse'>


200

In [149]:
request = twitter.request('users/lookup', {'screen_name':'realDonaldTrump'})
users = [r for r in request]
users['id']

TypeError: list indices must be integers or slices, not str

In [106]:
users = [u for u in request]
[u['id'] for u in users]

[25073877]

In [53]:
def get_users(twitter, screen_names):
    request = twitter.request('users/lookup', {'screen_name':screen_names})
    users = [r for r in request]
    return users

twitter = get_twitter('twitter.cfg')
users = get_users(twitter, read_screen_names('candidates.txt'))
print([u['id'] for u in users])

[111216929, 95713333, 1339835893, 25073877]


In [5]:
def get_friends(twitter, screen_name):
    request = twitter.request('friends/ids',{'screen_name':screen_name})
    friends = [r for r in request]
    return sorted(friends)
    
twitter = get_twitter('twitter.cfg')
get_friends(twitter, 'aronwc')[:5]

[695023, 1697081, 8381682, 10204352, 11669522]

In [54]:
def add_all_friends(twitter, users):
    for u in users:
        ids = get_friends(twitter,u['screen_name'])
        u['friends'] = ids
    
twitter = get_twitter('twitter.cfg')
users = get_users(twitter,['poojitha_vd','cowshik10'])
add_all_friends(twitter, users)
users[0]['friends'][:5]
    

[428333, 742143, 759251, 807095, 813286]

In [19]:
def print_num_friends(users):
    twitter = get_twitter('twitter.cfg')
    for u in users:
        frunds = get_friends(twitter,u['screen_name'])
        print(len(frunds))
        
users = [{'screen_name':'realDonaldTrump'}]
print_num_friends(users)

42


In [20]:
c = Counter()
def count_friends(users):
    twitter = get_twitter('twitter.cfg')
    add_all_friends(twitter,users)
    for u in users:
        c.update(u['friends'])
    return c
    
twitter = get_twitter('twitter.cfg')
users = get_users(twitter,['realDonaldTrump','HillaryClinton','DrJillStein','GovGaryJohnson'])
count_friends(users).most_common(5)
        
        

[(15846407, 3), (19608297, 3), (41877508, 2), (12, 2), (130613344, 2)]

In [49]:
import itertools
def friend_overlap(users):
    twitter = get_twitter('twitter.cfg')
    for u in users:
        f = get_friends(twitter,u['screen_name'])
#        print(list(itertools.product([u['screen_name']],[(len(f))])))
    print(type(f))
twitter = get_twitter('twitter.cfg')            
users = get_users(twitter,['realDonaldTrump','HillaryClinton','DrJillStein','GovGaryJohnson'])
friend_overlap(users)    

0
0
0
0


In [45]:
def friend_overlap(users):
    twitter = get_twitter('twitter.cfg')
    c = count_friends(users)
    print(c.most_common(5))
twitter = get_twitter('twitter.cfg')            
users = get_users(twitter,['realDonaldTrump','HillaryClinton','DrJillStein','GovGaryJohnson'])
friend_overlap(users)

TwitterRequestError: Twitter request failed (429)

In [56]:
twitter = get_twitter('twitter.cfg')
f1 = get_friends(twitter,'realDonaldTrump')
f2 = get_friends(twitter,'HillaryClinton')
mutual_friends = list(set(f1)&set(f2))
request = twitter.request('users/lookup',{'user_id':mutual_friends})
friend = [f for f in request]
print([f['screen_name'] for f in friend])

TwitterRequestError: Twitter request failed (429)

In [59]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
twitter = get_twitter('twitter.cfg')
users = get_users(twitter,['realDonaldTrump','HillaryClinton','DrJillStein','GovGaryJohnson'])
count = count_friends(users)
graph = nx.DiGraph()
graph.add_edge([users],str(len(count)))
nx.draw(graph, with_labels=True)

TwitterConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=5)

In [26]:
twitter = get_twitter('twitter.cfg')
request = twitter.request('friendships/lookup', {'screen_name': ['realDonaldTrump','HillaryClinton']})
foll = [print (r) for r in request]


{'connections': ['none'], 'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877'}


<class 'TwitterAPI.TwitterAPI.TwitterResponse'>


200

In [7]:
print([i*2 for i in [1,2,3]] )


[2, 4, 6]


In [15]:
request.status_code

401